# Test the Trend Index

## Check if index is up!

In [1]:
import requests
res = requests.get('http://foodmap.isti.cnr.it:9200', auth=('elastic', 'changeme'))
print(res.content)

{
  "name" : "loGu88Z",
  "cluster_name" : "foodmap-cluster",
  "cluster_uuid" : "u6p1OlcKS8K0ulvvGBdVKw",
  "version" : {
    "number" : "5.4.0",
    "build_hash" : "780f8c4",
    "build_date" : "2017-04-28T17:43:27.229Z",
    "build_snapshot" : false,
    "lucene_version" : "6.5.0"
  },
  "tagline" : "You Know, for Search"
}



In [2]:
from elasticsearch import Elasticsearch
es = Elasticsearch(['foodmap.isti.cnr.it'], http_auth=('elastic', 'changeme'), port=9200)
print es

<Elasticsearch([{u'host': u'foodmap.isti.cnr.it'}])>


## Index operations

#### Delete index, delete tweets by id and query

In [3]:
# es.indices.delete(index='trend', ignore=[400, 404])

{u'acknowledged': True}

In [ ]:
# es.delete_by_query(index='trend', doc_type='tweet', body={
#     'query': {
#         "match_all" : {}
#     }
# })

#### Count elements in index

In [12]:
es.count(index='trend', doc_type='tweet')

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5}, u'count': 211}

#### GET all docs from index

In [8]:
res = es.search(index="trend", doc_type='tweet', size=100, body={"query": {"match_all": {}}})
# [x['_source'] for x in res['hits']['hits']]
for r in res['hits']['hits'][:10]:
    print r
    print 

{u'_score': 1.0, u'_type': u'tweet', u'_id': u'840193402264309760', u'_source': {u'username': u'PeggySueCordoba', u'lang': u'en', u'text_categories': [u' pancakes'], u'city': u'C\xf3rdoba', u'text': u'Partners in crime... \U0001f52a #peggysues #peggysuescordoba #tortitas #pancakes #pancakestower #cordoba\u2026 https://t.co/rBnOlZr29e', u'img_categories': None, u'hashtags': [u'#peggysues', u'#peggysuescordoba', u'#tortitas', u'#pancakes', u'#pancakestower', u'#cordoba\u2026'], u'img_flag': True, u'timestamp_ms': u'1489152691795', u'bounding_box': {u'type': u'Polygon', u'coordinates': [[[-4.99701, 37.664255], [-4.99701, 38.030946], [-4.351492, 38.030946], [-4.351492, 37.664255]]]}, u'coords': [-4.78169551, 37.88771891], u'id_str': u'840193402264309760', u'country': u'Espa\xf1a', u'id': 840193402264309760, u'created_at_day': 20170310, u'created_at_datetime': u'2017-03-10T13:31:31', u'media_url': None}, u'_index': u'trend'}

{u'_score': 1.0, u'_type': u'tweet', u'_id': u'840193469863854080

#### GET specific tweets

In [15]:
es.get(index='trend', doc_type='tweet', id=880404048834416644)

{u'_id': u'880404048834416644',
 u'_index': u'stream',
 u'_source': {u'bounding_box': {u'coordinates': [[[120.989705, 14.589376],
     [120.989705, 14.776648],
     [121.135766, 14.776648],
     [121.135766, 14.589376]]],
   u'type': u'Polygon'},
  u'city': u'Quezon City',
  u'coords': [121.07463598, 14.63343778],
  u'country': u'Republic of the Philippines',
  u'created_at_datetime': u'2017-06-29T12:34:17',
  u'created_at_day': 20170629,
  u'hashtags': [u'#foodie', u'#foodporn'],
  u'id': 880404048834416644,
  u'id_str': u'880404048834416644',
  u'img_categories': None,
  u'img_flag': True,
  u'lang': u'en',
  u'media_url': None,
  u'text': u'donuts on stick. \U0001f369 #foodie #foodporn @ Kantorini Food Park https://t.co/OghzOaiJuK',
  u'text_categories': [u' donuts'],
  u'timestamp_ms': u'1498739657015',
  u'username': u'oshmacaroni'},
 u'_type': u'tweet',
 u'_version': 1,
 u'found': True}

In [19]:
results = es.search(index='trend', doc_type='tweet', size=100, body={
  'query': {
    'match': {
      'text_categories': 'donuts',
#       'day': 20170618,
#         'sort': [
#         {'committed_date': {'order': 'desc'}}
#       ],
     }
  }
})

print results
print len(results["hits"]["hits"])

{u'hits': {u'hits': [{u'_score': 0.6931472, u'_type': u'tweet', u'_id': u'880404048834416644', u'_source': {u'username': u'oshmacaroni', u'lang': u'en', u'text_categories': [u' donuts'], u'city': u'Quezon City', u'text': u'donuts on stick. \U0001f369 #foodie #foodporn @ Kantorini Food Park https://t.co/OghzOaiJuK', u'img_categories': None, u'hashtags': [u'#foodie', u'#foodporn'], u'img_flag': True, u'timestamp_ms': u'1498739657015', u'bounding_box': {u'type': u'Polygon', u'coordinates': [[[120.989705, 14.589376], [120.989705, 14.776648], [121.135766, 14.776648], [121.135766, 14.589376]]]}, u'coords': [121.07463598, 14.63343778], u'id_str': u'880404048834416644', u'country': u'Republic of the Philippines', u'id': 880404048834416644, u'created_at_day': 20170629, u'created_at_datetime': u'2017-06-29T12:34:17', u'media_url': None}, u'_index': u'stream'}], u'total': 1, u'max_score': 0.6931472}, u'_shards': {u'successful': 5, u'failed': 0, u'total': 5}, u'took': 2, u'timed_out': False}
1


#### TEST image classifier

In [ ]:
# test some of the urls from the indexed tweets
# by reclassifying it with the IMG classifier hosted on our machine
request_string = 'http://test.tripbuilder.isti.cnr.it:8080/FoodRecognition/services/IRServices/recognizeByURL?imgURL='
for tweet in results["hits"]["hits"]:
    img_url = tweet["_source"]["media_url"]
    res = requests.get(request_string + img_url)

    print tweet["_source"]["text"]
    print img_url
    print res.content
    print json.loads(res.text)["guessed"]
    print get_image_category(img_url)
    print 
    print

In [ ]:
# import requests
# import json

# response = requests.get('http://test.tripbuilder.isti.cnr.it:5000/sync')
# json_data = json.loads(response.text)
# json_data